
# Innomatics Research Labs – Advanced GenAI Internship
## Food Delivery Data Integration & Analysis

This notebook demonstrates end-to-end data integration and analysis of a food delivery dataset. 
It combines transactional, user, and restaurant data from multiple formats to create a single source of truth.

All insights are derived exclusively from the final merged dataset, ensuring data consistency and reproducibility.


In [ ]:

import pandas as pd
import sqlite3


In [ ]:

orders = pd.read_csv('orders.csv')
orders.head()


In [ ]:

users = pd.read_json('users.json')
users.head()


In [ ]:

# Create SQLite in-memory database and read the SQL file
conn = sqlite3.connect(':memory:')
with open('restaurants.sql', 'r') as f:
    sql_script = f.read()
conn.executescript(sql_script)

# Load restaurants table into pandas
restaurants = pd.read_sql('SELECT * FROM restaurants', conn)
restaurants.head()


In [ ]:

# Merge orders + users on user_id
merged1 = pd.merge(orders, users, on='user_id', how='left')

# Merge with restaurants on restaurant_id
final_df = pd.merge(merged1, restaurants, on='restaurant_id', how='left')

final_df.head()


In [ ]:

final_df.to_csv('final_food_delivery_dataset.csv', index=False)


In [ ]:

# 1. Total orders by Gold members
gold_orders = final_df[final_df['membership']=='Gold']
total_gold_orders = gold_orders.shape[0]

# 2. Total revenue from Hyderabad
total_revenue_hyd = final_df[final_df['city']=='Hyderabad']['total_amount'].sum()

# 3. Distinct users who placed at least one order
distinct_users = final_df['user_id'].nunique()

# 4. Average order value for Gold members
avg_order_value_gold = round(gold_orders['total_amount'].mean(), 2)

# 5. Orders for restaurants with rating >= 4.5
orders_rating_45 = final_df[final_df['rating'] >= 4.5].shape[0]

# 6. Orders in top revenue city among Gold members
gold_city_revenue = gold_orders.groupby('city')['total_amount'].sum()
top_gold_city = gold_city_revenue.idxmax()
orders_top_gold_city = gold_orders[gold_orders['city']==top_gold_city].shape[0]

# Display results
print("✅ Numerical Answers")
print("Total orders by Gold members:", total_gold_orders)
print("Total revenue from Hyderabad:", int(total_revenue_hyd))
print("Distinct users who placed orders:", distinct_users)
print("Average order value (Gold members):", avg_order_value_gold)
print("Orders with rating >= 4.5:", orders_rating_45)
print("Orders in top Gold revenue city:", orders_top_gold_city)
print("Top revenue city among Gold members:", top_gold_city)



# Fill-in-the-Blank Answers

- Column used to join orders.csv and users.json: **user_id**  
- Dataset containing cuisine and rating info: **SQL format**  
- Total rows in final dataset: **10,000**  
- If a user has no matching record in users.json: **NaN**  
- Pandas function to combine datasets: **merge()**  
- Column membership in final dataset originates from: **users.json**  
- Join key for orders + restaurants: **restaurant_id**  
- Column identifying type of food: **cuisine**  
- If a user places multiple orders, their details appear: **multiple**



# Methodology

1. **Load datasets** from multiple formats: CSV, JSON, SQL.
2. **Merge datasets** using left joins to preserve all transactional records.
3. **Create final dataset**: `final_food_delivery_dataset.csv`.
4. **Calculate numerical metrics** using the final dataset.
5. **Document all answers** and fill-in-the-blank responses.
6. **Ensure reproducibility**: all analysis is derived from final dataset only.
